Bayesian interpretation of medical tests
-----------------------------------------

An example that demonstrates a simple hierarchical model.

Copyright 2016 Allen Downey

MIT License: http://opensource.org/licenses/MIT

In [1]:
from __future__ import print_function, division

import thinkplot
from thinkbayes2 import Pmf, Suite

import numpy as np

%matplotlib inline

Suppose we test a patient to see if they have a disease, and the test comes back positive.  What is the probability that the patient is actually sick (that is, has the disease)?

To answer this question, we need to know:

*  The prevalence of the disease in the population the patient is from.  Let's assume the patient is identified as a member of a population where the prevalence is `p`.

*  The sensitivity of the test, `s`, which is the probability of a positive test if the patient is sick.

*  The false positive rate of the test, `t`, which is the probability of a positive test if the patient is not sick.

Given these parameters, we can compute the probability that the patient is sick, given a positive test.

### Test class

To do that, I'll define a `Test` class that extends `Suite`, so it inherits `Update` and provides `Likelihood`.

The instance variables of `Test` are:

*  `p`, `s`, and `t`: Copies of the parameters.
*  `d`: a dictionary that maps from hypotheses to their probabilities.  The hypotheses are the strings `sick` and `notsick`.
*  `likelihood`: a dictionary that encodes the likelihood of the possible data values `pos` and `neg` under the hypotheses.



In [2]:
class Test(Suite):
    """Represents beliefs about a patient based on a medical test."""
    
    def __init__(self, p, s, t):
        # store the parameters
        self.p = p
        self.s = s
        self.t = t
        
        # initialize the prior probabilities
        d = dict(sick=p, notsick=1-p)
        super(Test, self).__init__(d)
        
        # make a nested dictionary to compute likelihoods
        self.likelihood = dict(pos=dict(sick=s, notsick=t),
                               neg=dict(sick=1-s, notsick=1-t))
        
    def Likelihood(self, data, hypo):
        """
        data: 'pos' or 'neg'
        hypo: 'sick' or 'notsick'
        """
        return self.likelihood[data][hypo]

Now we can create a `Test` object with parameters chosen for demonstration purposes (most medical tests are better than this!):

In [3]:
p = 0.1      # prevalence
s = 0.9      # sensitivity
t = 0.3      # false positive rate
test = Test(p, s, t)
test.Print()

notsick 0.9
sick 0.1


If you are curious, here's the nested dictionary that computes the likelihoods:

In [4]:
test.likelihood

{'neg': {'notsick': 0.7, 'sick': 0.09999999999999998},
 'pos': {'notsick': 0.3, 'sick': 0.9}}

And here's how we update the `Test` object with a positive outcome:

In [5]:
test.Update('pos')
test.Print()

notsick 0.7499999999999999
sick 0.24999999999999997


The positive test provides evidence that the patient is sick, increasing the probability from 0.1 to 0.25.

### Hierarchical model

So far, this is basic Bayesian inference.  Now let's add a wrinkle.  Suppose that we don't know the value of `t` with certainty, but we have reason to believe that `t` is either 0.2 or 0.4 with equal probability.

We can represent this belief with a hierarchical model, where the levels of the hierarchy are:

*  At the top level, the possible values of `t` and their probabilities.
*  At the bottom level, the probability that the patient is sick or not, conditioned on `t`.

To represent the hierarchy, I'll define a `MetaTest`, which is a `Suite` that contains `Test` objects with different values of `t` as hypotheses.

In [6]:
class MetaTest(Suite):
    """Represents a set of tests with different values of `t`."""
    
    def Likelihood(self, data, hypo):
        """
        data: 'pos' or 'neg'
        hypo: Test object
        """
        # the return value from `Update` is the total probability of the
        # data for a hypothetical value of `t`
        return hypo.Update(data)
    
    def Print(self):
        for test, prob in self.Items():
            print('t=', test.t, test, prob)

To update a `MetaTest`, we update each of the hypothetical `Test` objects.  The return value from `Update` is the normalizing constant, which is the total probability of the data under the hypothesis.

We use the normalizing constants from the bottom level of the hierarchy as the likelihoods at the top level.

Here's how we create the `MetaTest` for the scenario we described:

In [7]:
q = 0.5
t1 = 0.2
t2 = 0.4

test1 = Test(p, s, t1)
test2 = Test(p, s, t2)

metatest = MetaTest({test1:q, test2:1-q})
metatest.Print()

t= 0.2 Test({'notsick': 0.9, 'sick': 0.1}) 0.5
t= 0.4 Test({'notsick': 0.9, 'sick': 0.1}) 0.5


At the top level, there are two tests, with different values of `t`.  Initially, they are equally likely.

When we update the `MetaTest`, it updates the embedded `Test` objects and then the `MetaTest` itself.

In [8]:
metatest.Update('pos')

0.36000000000000004

Here are the results.

In [9]:
metatest.Print()

t= 0.2 Test({'notsick': 0.6666666666666666, 'sick': 0.3333333333333333}) 0.37499999999999994
t= 0.4 Test({'notsick': 0.7999999999999999, 'sick': 0.19999999999999998}) 0.625


Because a positive test is more likely if `t=0.4`, the positive test is evidence in favor of the hypothesis that `t=0.4`.

This `MetaTest` object represents what we should believe about `t` after seeing the test, as well as what we should believe about the probability that the patient is sick.

### Predictive probabilities

To compute the probability that the patient is sick, we have to compute the predictive probabilities of `sick` and `notsick`, averaging over the possible values of `t`.  The following function computes this distribution:

In [10]:
def MakeMixture(metapmf, label='mix'):
    """Make a mixture distribution.

    Args:
      metapmf: Pmf that maps from Pmfs to probs.
      label: string label for the new Pmf.

    Returns: Pmf object.
    """
    mix = Pmf(label=label)
    for pmf, p1 in metapmf.Items():
        for x, p2 in pmf.Items():
            mix.Incr(x, p1 * p2)
    return mix

Here's the posterior predictive distribution:

In [11]:
predictive = MakeMixture(metatest)
predictive.Print()

notsick 0.7499999999999999
sick 0.24999999999999994


After seeing the test, the probability that the patient is sick is 0.25, which is the same result we got with `t=0.3`.  So at first glance it seems like we might not need the hierarchical model at all.  Maybe we could have computed the mean of `t` and done a simple update.

### Two patients

If the goal is to make a single prediction for a single patient, that's true.  But let's try a different scenario.  Suppose you test two patients and they both test positive.  What is the probability that they are both sick?

To answer that, I define a few more functions to work with Metatests:

In [12]:
def MakeMetaTest(p, s, pmf_t):
    """Makes a MetaTest object with the given parameters.
        
    p: prevalence
    s: sensitivity
    pmf_t: Pmf of possible values for `t`
    """
    tests = {}
    for t, q in pmf_t.Items():
        tests[Test(p, s, t)] = q
    return MetaTest(tests)

def Marginal(metatest):
    marginal = Pmf()
    for test, prob in metatest.Items():
        marginal[test.t] = prob
    return marginal

def Conditional(metatest, t):
    for test, prob in metatest.Items():
        if test.t == t:
            return test, prob

`MakeMetaTest` makes a `MetaTest` object starting with a given PMF of `t`.

`Marginal` extracts the PMF of `t` from a `MetaTest`.

`Conditional` takes a specified value for `t` and returns the probabilities of `sick` and `notsick` conditioned on `t`.

I'll test it out using the same parameters from above:

In [13]:
q = 0.5
t1 = 0.2
t2 = 0.4
pmf_t = Pmf({t1:q, t2:1-q})
pmf_t.Print()

0.2 0.5
0.4 0.5


Here are the results

In [14]:
metatest = MakeMetaTest(p, s, pmf_t)
metatest.Update('pos')
metatest.Print()

t= 0.2 Test({'notsick': 0.6666666666666666, 'sick': 0.3333333333333333}) 0.37499999999999994
t= 0.4 Test({'notsick': 0.7999999999999999, 'sick': 0.19999999999999998}) 0.625


Same as before.  And we can extract the posterior distribution of `t`.

In [15]:
Marginal(metatest).Print()

0.2 0.37499999999999994
0.4 0.625


Having seen one positive test, we are a little more inclined to believe that `t=0.4`; that is, that the false positive rate is high.

And we can extract the conditional distributions for the patient:

In [16]:
cond1, prob = Conditional(metatest, t1)
cond1.Print()

notsick 0.6666666666666666
sick 0.3333333333333333


In [17]:
cond2, prob = Conditional(metatest, t2)
cond2.Print()

notsick 0.7999999999999999
sick 0.19999999999999998


Finally, we can make a predictive distribution for the patient, which is a weighted mixture of the conditional distributions:

In [18]:
predictive = MakeMixture(metatest)
predictive.Print()

notsick 0.7499999999999999
sick 0.24999999999999994


At this point we have a `MetaTest` that contains our updated information about the test (the distribution of `t`) and about the patient that tested positive.

### The second patient

Now suppose the second patient arrives.  We need a new `MetaTest` that contains the updated information about the test, but no information about the patient other than the prior proability of being sick, `p`:

In [19]:
metatest1 = MakeMetaTest(p, s, Marginal(metatest))
metatest1.Print()

t= 0.2 Test({'notsick': 0.9, 'sick': 0.1}) 0.37499999999999994
t= 0.4 Test({'notsick': 0.9, 'sick': 0.1}) 0.625


Now we can update this `MetaTest` with the result from the second test:

In [20]:
metatest1.Update('pos')
metatest1.Print()

t= 0.2 Test({'notsick': 0.6666666666666666, 'sick': 0.3333333333333333}) 0.2647058823529411
t= 0.4 Test({'notsick': 0.7999999999999999, 'sick': 0.19999999999999998}) 0.7352941176470589


This distribution contains updated information about the test, based on two positive outcomes, and updated information about a patient who has tested positive (once).

In [21]:
predictive = MakeMixture(metatest1)
predictive.Print()

notsick 0.7647058823529411
sick 0.23529411764705882


Now, to compute the probability that both patients are sick, we have to know the distribution of `t` for both patients.  And that depends on details of the scenario.  There are two possibilities:

*  Scenario A: Suppose there are two different tests, with different false positive rates, and we don't know which tests were used.  Or suppose that the false positive rate is different for different groups of people, and we don't know which group the patients are from.  In either case, the values of `t` for the two patients are independent.

*  Scenario B: Alternatively, suppose there is only one test and we have reason to believe that the false positive rate is the same for everyone, so `t` is either `t1` for everyone or `t2` for everyone, but we don't know which.  In that case, the value of `t` for the two patients is the same.

The probability that both patients are sick is different in these two scenarios.  I'll compute both.

### Scenario A

In Scenario A, we can compute the probability of `sick` and `nonsick` independently for the two patients and then compute the "sum" of those distributions:

In [22]:
conjunction = predictive + predictive
conjunction.Print()

notsicknotsick 0.5847750865051903
notsicksick 0.17993079584775085
sicknotsick 0.17993079584775085
sicksick 0.05536332179930796


In [23]:
predictive['sick']**2

0.05536332179930796

In Scenario A, the probability that both patients are sick is just the square of the probability that each patient is sick.

### Scenario B

In Scenario B we have to work a little harder.  For a given value of `t` we can compute the probability of `sicksick` by conjoining the conditional distributions for the two patients:

In [24]:
cond_t1, p_t1 = Conditional(metatest1, t1)
conjunction_t1 = cond_t1 + cond_t1
print(p_t1)
conjunction_t1.Print()

0.2647058823529411
notsicknotsick 0.4444444444444444
notsicksick 0.2222222222222222
sicknotsick 0.2222222222222222
sicksick 0.1111111111111111


If we know that `t=t1`, the probability of `sicksick` is 0.111.  And for `t=t2`:

In [25]:
cond_t2, p_t2 = Conditional(metatest1, t2)
conjunction_t2 = cond_t2 + cond_t2
print(p_t2)
conjunction_t2.Print()

0.7352941176470589
notsicknotsick 0.6399999999999999
notsicksick 0.15999999999999998
sicknotsick 0.15999999999999998
sicksick 0.039999999999999994


If we know that `t=t2`, the probability of `sicksick` is 0.04.

The overall probability of `sicksick` is the weighted average of these probabilities:

In [26]:
p_t1 * conjunction_t1['sicksick'] + p_t2 * conjunction_t2['sicksick']

0.05882352941176469

We can generalize that process to compute all of the posterior probabilities.

The following function computes the conjunctions:

In [27]:
def MakeConjunction(metatest, t):
    cond, p = Conditional(metatest, t)
    return cond + cond, p

Here's an example that computes the conjunction for `t=t1`:

In [28]:
pmf, prob = MakeConjunction(metatest1, t1)
print(prob)
pmf.Print()

0.2647058823529411
notsicknotsick 0.4444444444444444
notsicksick 0.2222222222222222
sicknotsick 0.2222222222222222
sicksick 0.1111111111111111


In [29]:
Marginal(metatest1)

Pmf({0.2: 0.2647058823529411, 0.4: 0.7352941176470589})

Now we can make a MetaTest that contains the conjunctions with the right weights:

In [30]:
metapmf = Pmf()
for t in Marginal(metatest1):
    pmf, prob = MakeConjunction(metatest1, t)
    metapmf[pmf] = prob
    
metapmf.Print()

Pmf({'notsicksick': 0.15999999999999998, 'sicknotsick': 0.15999999999999998, 'sicksick': 0.039999999999999994, 'notsicknotsick': 0.6399999999999999}) 0.7352941176470589
Pmf({'notsicksick': 0.2222222222222222, 'sicknotsick': 0.2222222222222222, 'sicksick': 0.1111111111111111, 'notsicknotsick': 0.4444444444444444}) 0.2647058823529411


And finally we can use `MakeMixture` to compute the weighted averages of the posterior probabilities:

In [31]:
predictive = MakeMixture(metapmf)
predictive.Print()

notsicknotsick 0.588235294117647
notsicksick 0.1764705882352941
sicknotsick 0.1764705882352941
sicksick 0.05882352941176469


### Summary

In summary:

*  In Scenario A, the values of `t` for the two patients are independent, and the probability that both are sick is xxx.

*  In Scenario B, the value of `t` has to be the same for both patients, and the probability that both are sick is xxx.

A real scenario might combine elements of both; that is, the false positive rate might be different for different people, and we might have some uncertainty about what it is.  In that case, the most accurate predictive probability might be anywhere between the values we computed.